## Properties

It's a special sort of object attribute; almost a cross between a method and an attritube. 

In [1]:
#example

class Person:
    def __init__(self, firstname, lastname):
        self.firstname = firstname
        self.lastname = lastname
    @property
    def fullname(self):
        return self.firstname + " " + self.lastname

In [2]:
joe = Person("Joe","Smith")
joe.fullname

'Joe Smith'

In [3]:
joe.fullname()

TypeError: 'str' object is not callable

As defined above, fullname is ***read-only***. we can't modify it. 

In [4]:
joe.fullname = "Joseph Smith"

AttributeError: can't set attribute 'fullname'

In other word, Python properties are read-only by default.<br>
Another way of saying this is that @property dedfines a getter, but not a setter.

## Setter

In [5]:
class Person:
    def __init__(self, firstname, lastname):
        self.firstname = firstname
        self.lastname = lastname
    @property
    def fullname(self):
        return self.firstname + " " + self.lastname
    @fullname.setter
    def fullname(self, value):
        self.firstname, self.lastname = value.split(" ", 1)

In [6]:
joe = Person("Joe","Smith")
joe.firstname

'Joe'

In [8]:
joe.lastname

'Smith'

In [9]:
joe.fullname = "Joseph Smith"
joe.firstname

'Joseph'

You may be wondering "Why is 'fullname' defined twice? and why is the second decorator named @fullname.setter?" <br><br>

The code is actually correct. The following must come first 
    
    @property 
    def fullname():

Craeting property means that an object named the function name will exist in the namespace of the class, and it has a method named @function_name.setter 
<br><br>

Properties enable a useful collection of design patterns. One is in creating read-only member variables. <br><br>

It's also common to have the property backed by a single, non-public member variable. See the example: 


In [10]:
class Coupon:
    def __init__(self,amount):
        self._amount=amount
    @property
    def amount(self):
        return self._amount

In Python, prefixing a member variable by a single underscore signals the variable is non-public, i.e.

it should only be accessed internally, inside methods of that class, or its subclasses.

What this pattern says is "you can access this variable, but not change it".

### UseCase 1 - ticketing

Suppose my event-management application has a Ticket class, representing tickets sold to concert-goers: 

In [11]:
class Ticket:
    def __init__(self, price):
        self.price = price
# And some other methods...

One day, we find a bug in our web UI, which lets some shifty customers adjust the price to a negative value… so we ended up actually paying them to go to the concert. Not good!<br><br>

The first priority is, of course, to fix the bug in the UI. 
But how do we modify our code to prevent this from ever happening again? <br><br>

Before reading further, look at the Ticket class and ponder - how could you use properties to make this kind of bug impossible in the future? The answer: verify the new price is non-zero in the setter:

In [12]:
class Ticket:
    def __init__(self,price):
        self._price = price
    
    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self,new_price):
        if new_price<0:
            raise ValueError("Nice try")
        self._price = new_price
    
        

In [13]:
t = Ticket(42)
t.price

42

In [14]:
t.price = -1

ValueError: Nice try

#### However...
There is a defect in this new Ticket class.<br><br>
The problem is that we CAN'T CHANGE the price to a negative value, But CAN CREATE a ticket with a negative price to begin with.

The solution is to use the setter in the constructor instead. 


In [15]:
class Ticket:
    def __init__(self, price):
    # instead of "self._price = price"
        self.price = price
        
    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self, new_price):
    # Only allow positive prices.
        if new_price < 0:
            raise ValueError("Nice try")
        self._price = new_price

In [16]:
a = Ticket(42)
a.price

42

Wait, "a.price" is supposed to call property but property returns self._price...<br>

At initialization, we didn't put self._price..?
<br>

Yes, you can reference self.price in methods of the class. When we write :

    self.price = price

Python translates this to calling *price setter*. So it means ***self.price here refers to price setter method.***<br>

So this final version of Ticket centralizes all reads AND writes of self._price.<br>

It's a useful encapsulation principle in general. 

In [17]:
b= Ticket(-4)

ValueError: Nice try

### Properties and Refactoring

Properties are important in most Languages today. Imagine writing a simple money class. 

In [18]:
class Money:
    def __init__(self,dollars,cents):
        self.dollars = dollars
        self.cents =cents 

Suppose you put this class in a library, which many developers are using. People on your current team, perhaps developers on different teams. Or maybe you release it as open-source, so developers around the world use and rely on this class.<br><br>

Now, you realize many of Money's methods can be simpler and more straightforward if they operate on the total number of cents rather than dollar and cents separately. So you ***refactor*** the internal state: <br>


In [19]:
class Money:
    def __init__(self,dollars,cents):
        self.total_cents = dollars *100 + cents

This minor change creates a MAJOR maintainability issue.<br>

Here is the trouble: your original Money has dollars and cents.<br>

And since many developers already use them, changing to total_cents break all their code. <br>

SO what do you do? 

You want to two things to happen:
- Use ***total_cents*** internally
- All code using dollars and cents continues to work, without modification.



In [20]:
class Money:
    def __init__(self,dollars,cents):
        self.total_cents= dollars * 100 +cents 
    
    #for dolloars
    @property
    def dollars(self):
        return self.total_cents //100
    
    @dollars.setter
    def dollars(self,new_dollars):
        self.total_cents = 100*new_dollars + self.cents
    
    #for cents
    @property
    def cents(self):
        return self.total_cents %100
    @cents.setter
    def cetns(self,new_cents):
        self.total_cents= 100 * self.dollars + new_cents

In [21]:
money = Money(27,12)
money.total_cents

2712

In [22]:
money.cents

12

In [25]:
money.dollars=35
money.total_cents

3512